In [13]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import param

In [2]:
all_sui_reg = pd.read_csv('Data/all_suicides/suicidios-reg.csv')
print(all_sui_reg)

    cod;nome;período;valor
0           3;SE;1989;1971
1             1;N;1989;220
2            5;CO;1989;349
3            2;NE;1989;540
4            4;S;1989;1411
..                     ...
150         2;NE;2019;3082
151         5;CO;2019;1283
152         3;SE;2019;4930
153          1;N;2019;1058
154          4;S;2019;3167

[155 rows x 1 columns]


In [3]:
# All of the files are csv, with separators ';'
# Gonna try to adjust our Df accordingly

all_sui_reg = pd.read_csv('Data/all_suicides/suicidios-reg.csv',sep=';')
print(all_sui_reg)

     cod nome  período  valor
0      3   SE     1989   1971
1      1    N     1989    220
2      5   CO     1989    349
3      2   NE     1989    540
4      4    S     1989   1411
..   ...  ...      ...    ...
150    2   NE     2019   3082
151    5   CO     2019   1283
152    3   SE     2019   4930
153    1    N     2019   1058
154    4    S     2019   3167

[155 rows x 4 columns]


In [11]:
# We need to adjust every df to plot them later on, and to have easier access to the data
# For now we are going to adjust the regions, states(UFs), Men and Womens data.

all_sui_uf = pd.read_csv('Data/all_suicides/suicidios-uf.csv',sep=';')
men_sui_reg = pd.read_csv('Data/men/suicidios-de-homens-reg.csv',sep=';')
wom_sui_reg = pd.read_csv('Data/women/suicidio-de-mulheres-reg.csv',sep=';')
rate_sui_reg = pd.read_csv('Data/rate/taxa-reg.csv',sep=';')
all_sui_reg = pd.read_csv('Data/all_suicides/suicidios-reg.csv',sep=';')

In [8]:
# Let's take a look at how we can filter data
print(wom_sui_reg)

    cod nome  período  valor
0     5   CO     2000    157
1     1    N     2000     84
2     2   NE     2000    261
3     3   SE     2000    499
4     4    S     2000    381
..  ...  ...      ...    ...
95    1    N     2019    231
96    4    S     2019    653
97    3   SE     2019   1117
98    2   NE     2019    616
99    5   CO     2019    302

[100 rows x 4 columns]


In [23]:
'''
--------------------Explaining this df--------------------
nome -> name of the region (abbreviated)
período -> year 
valor -> 'value', number of suicides
Rate data value per 100.000 population
'''

"\n\n--------------------Explaining this df.--------------------\nnome -> name of the region (abbreviated)\nperíodo -> year \nvalor -> 'value', number of suicides\n\n"

In [6]:
# We can separate in regions this way
rate_sui_reg[rate_sui_reg['nome']=='CO']

,cod,nome,período,valor
0,5,CO,1980,2.94
5,5,CO,1981,3.32
10,5,CO,1982,3.32
15,5,CO,1983,3.46
20,5,CO,1984,3.41
25,5,CO,1985,3.42
30,5,CO,1986,3.28
35,5,CO,1987,4.30
40,5,CO,1988,4.00
45,5,CO,1989,3.90


## Defining Widgets 

In [75]:
# Define year slider

year_slider = pn.widgets.IntSlider(name= 'Year slider',
                                   start= 2000, #lower bound year
                                   end = 2019,  #upper bound year
                                   step = 1,
                                   value = 2000 #default value
                                  )
year_slider

IntSlider(end=2019, name='Year slider', start=2000, value=2000)

In [12]:
# Checkboxes for women x men x total

# checkbox_group = pn.widgets.CheckBoxGroup(
#     name='Y axis', 
#     value=[wom_sui_reg['valor'],men_sui_reg['valor'],all_sui_reg['valor']], 
#     options=['Women', 'Men', 'Total'],
#     inline=True)

# checkbox_group

CheckBoxGroup(inline=True, options=['Women', 'Men', 'Total'])

In [94]:
# Radio buttons for women stats
# yaxis_wom_reg = pn.widgets.RadioButtonGroup(name='Y axis',
#                                        options = ['CO','SE','S','N','NE'],
#                                        button_type= 'success'
#                                            )
# yaxis_wom_reg


'''OPTIONS MUST BE CCOLUMNS, trying to make a big data frame with
both women, men, and total stats.'''

RadioButtonGroup(button_type='success', name='Y axis', options=['CO', 'SE', 'S', ...], value='CO')

In [101]:
idf = wom_sui_reg.interactive()

In [112]:
# Connect data pipeline with widgets

regions = ['CO','SE','S','N','NE']
wom_men_pipeline = (
    idf[
        (idf.período <= year_slider) &
        (year_slider.value_throttled == idf.período)
       
    ]
    .groupby(['nome','período']).sum()
    .reset_index()
    .sort_values(by='nome')  
    .reset_index(drop=True)
)


In [113]:
wom_men_pipeline